In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import numpy as np
import datetime as dt
import pickle

path_open = '/mnt/Z/УБПиАЭ/ОАиП/Оперативная работа/Nord Pool/Dash_daily and other/Clear data'; #источник данных

t_day = dt.datetime.now(); #Текущее дата-время
t_day = dt.datetime.combine(t_day.date(), dt.time(0, 0, 0)); #Для текущей даты укажем время 00-00-00

#Укажем окно дат для которого строится первоначальный график
l_day = t_day-dt.timedelta(25);
r_day = t_day+dt.timedelta(5);

long_table = 12; #Период на который формируется таблица со средневзвешенными ценами

visible_elem = ['Финляндия', 'Nasdaq', 'Переток NordPool']; #Элементы, которые отобразятся при открытии дашборда

In [2]:
# #CD296A Запасной цвет для тренда
line_color = {'Финляндия': '#CD2929', 'Норвегия': '#2939CC', 'Швеция': '#2990CC', 'Дания': '#29AECC',
              'Латвия': '#6A29CD', 'Литва': '#AC29CD', 'Эстония': '#CD29AC',
              'Германия': '#29CC6A', 'Франция': '#29CC2E', 'Бельгия': '#98CC29', 'Нидерланды': '#CCBC29', 'Австрия': '#29CDAC',
              'Польша': '#9A5ED6', 'Швейцария': '#5E9CD6',
              'NordPool': '#999', 
              'Nasdaq': '#CD2929'};

def load_data():
    """
    Загрузка данных происводится каждый раз при вызове функции при обновлении дашборда.
    Полученные результаты - глобальные переменные.
    """
    global price_day_data, nasdaq_day_data, exc_day_data, price_month_data, idx
    
    with open(path_open + '/All_data.pickle', 'rb') as f:
        data_out = pickle.load(f); #Загрузим данные
    
    day_data = data_out[1];
    month_data = data_out[2];
    
    price_day_data = day_data['Цена'];
    idx = price_day_data.index;
    
    nasdaq_day_data = day_data['Nasdaq'];
    exc_day_data = day_data['Переток'];
    
    price_month_data = month_data['Цена'];
    price_month_data.index = price_month_data.index.strftime('%m.%y'); #Переведем формат даты в удобный для просмотра
    price_month_data = price_month_data.applymap(lambda x: '{:.2f}'.format(x) if str(x)!='nan' else ''); #Отформатируем числа
    price_month_data = price_month_data.applymap(lambda x: x.replace('.', ',')); #Заменим разделить на привчный
    
    create_graph(idx.get_loc(l_day), idx.get_loc(r_day));
    
    return;

def create_graph(l_, r_):
    """
    Соберем нужные данные.
    Если мы обратимся без указания позиций ползунков, то они установятся -25 слева и + 5дней справа.
    """
    global data_graph, l_gran, r_gran
    l_gran, r_gran = l_, r_;
                 
    #Заполним конструктор графика
    data_graph = [];
    data_graph += generate_graph(price_day_data, '', 'line', '', 'y2', '€ %{y:.2f}'); #Посуточные цены
    data_graph += generate_graph(nasdaq_day_data, '', 'line', 'dash', 'y2', '€ %{y:.2f}'); #Прогноз Nasdaq
    data_graph += generate_graph(exc_day_data, 'Переток ', 'bar', '', 'y1', '%{y:.0f} MW');#Посуточный переток
    
    #Укажем какие данные будут отображены при открытие дашборда
    for elem_gr in data_graph:
        if elem_gr['name'] in visible_elem:
            elem_gr['visible'] = 'True';
            
    return;
            
def generate_graph(day_fr, prefix, line_type, line_dash, yaxis, txt_style):
    """
    Данные для построения посуточных графиков.
    На входе фрейм данных.
    Словарь с описанием типов линий и набором данных.
    """
    day_fr = day_fr.iloc[l_gran:r_gran];
    day_idx = day_fr.index;
    new_dan_gr = [];
    for i in day_fr.columns:
        new_dan_gr.append({'x': list(day_idx.astype(str).values), #list(day_idx.astype(str).values),
                           'y': list(day_fr[i].values),
                           'type': line_type,
                           'mode': '',
                           'name': prefix+i,
                           'hovertemplate': txt_style,
                           'visible': 'legendonly',
                           'showlegend': False,
                           'yaxis' : yaxis,
                           'line': {'color': line_color[i],
                                    'dash': line_dash},
                           'marker': {'color': line_color[i],
                                      'opacity': 0.5}
                           });
        
    return new_dan_gr;

In [3]:
#Фон
set_colors = {'background': '#FFFFFF',
              'text': '#000000',
              'graph_plot_bgcolor': '#F5F5F5',
              'graph_bgcolor': '#F5F5F5',
              'backgroundColor': '#F5F5F5'};

def nazv():
    """
    Название дашборда.
    """
    
    return html.H1(children = 'NordPool Dashboard',
                   style = {'textAlign': 'center',
                            'color': set_colors['text']}
                  );

In [4]:
#Формат осей 0y
y_ax = {'yaxis2': {'title': 'Цена, €/МВт⋅ч',
                   'tickprefix': '',
                   'overlaying': 'y',
                   'side': 'left',
                   'range': ''},
        'yaxis1': {'title': 'Переток, МВт',
                    'tickprefix': '',
                    'side': 'right',
                    'range': ''}
           };

def graph():
    """
    Графическая подложка.
    """
    
    return dcc.Graph(id = 'plot',
                     figure = {'data': data_graph,
                               'layout': {'title': {'text': 'Динамика посуточной цены покупки электроэнергии в Европе',
                                                    'xanchor': 'center'},
                                          'yaxis2': y_ax['yaxis2'],
                                          'yaxis1': y_ax['yaxis1'],
                                          'plot_bgcolor': set_colors['graph_plot_bgcolor'],
                                          'paper_bgcolor': set_colors['graph_bgcolor'],
                                          'font': {'color': set_colors['text']},
                                         },
                              }
                    );

In [5]:
def slider():
    """
    Дереаво для отображения ползунка.
    """
    pos = idx.get_loc(t_day); #Определим позицию текущего дня
    
    return html.P([html.Label(""),
                   dcc.RangeSlider(id = 'slider',
                                   marks = {idx.get_loc(i): str(i)[:10] #В качестве ключа должен быть номер элемента по порядку
                                            for x,i in enumerate(idx)
                                            if str(i)[5:10]=='01-01'}, #Отобразим на графике только первое число января каждого года
                                   min = 0,
                                   max = len(idx)-1,
                                   step = 1,
                                   value = [pos-25, pos+5]) #Отобразим 25 дней до и 5 после текущего дня
                  ],
                  style = {'width' : '94%',                                                     
                           'fontSize' : '20px',
                           'padding-left' : '2%',
                           'display': 'inline-block'}
                 );

In [6]:
def country():
    """
    Дерево мультивыбора отображения тренда цены на графике.
    """
    
    return html.P([html.Label('Выберите страну'),
                   dcc.Dropdown(id='country',
                                options = [{"label": i, "value": i} for i in price_day_data.columns.to_list()+nasdaq_day_data.columns.to_list()],
                                value=['Финляндия', 'Nasdaq'],
                                multi=True)
                  ], style = {'width': '400px',
                              'fontSize' : '20px',
                              'padding-left' : '50px',
                              'display': 'inline-block'}
                 );

In [7]:
#Состав регионов для группировки
sostav_reg = {'Север': ['Финляндия', 'Норвегия', 'Швеция', 'Дания'],
              'Центр': ['Франция', 'Швейцария', 'Германия', 'Австрия', 'Бельгия', 'Нидерланды', 'Польша'],
              'Балтия': ['Латвия', 'Литва', 'Эстония']
             };

def region():
    """
    Дерево для выпадающего списка с группами стран.
    """
    
    return html.P([html.Label("Выберете регион (опционально)"),
                   dcc.Dropdown(id='region',
                                options = [{'label': 'Север', 'value': 'Север'},
                                           {'label': 'Центр', 'value': 'Центр'},
                                           {'label': 'Балтия', 'value': 'Балтия'},
                                           {'label': 'Все', "value": 'Все'}])
                  ], style = {'width': '400px',
                              'fontSize' : '20px',
                              'padding-left' : '50px',
                              'display': 'inline-block'}
                 );

In [8]:
def peretok():
    """
    Возможность выбрать в каком направлении отображать переток.
    """
    peretok_dic = [{'label': 'Россия - '+i, 'value': 'Переток '+i} for i in exc_day_data.columns];
    
    return html.P([html.Label('Выберите направление перетока'),
                   dcc.Dropdown(id='peretok',
                                options=peretok_dic,
                                value=['Переток NordPool'],
                                multi=True)
                  ],  style = {'width': '400px',
                               'fontSize' : '20px',
                               'padding-left' : '50px',
                               'display': 'inline-block'
                              }
                 );

In [9]:
def naz_tabl():
    """
    Название таблицы средневзвешенной цены.
    """
    
    return html.H4(children='Средневзвешенная цена, €/МВт⋅ч',
                   style={'height': '3px',
                          'fontSize' : '20px',
                          'padding-left' : '125px'}
                  );

In [10]:
#Форматирование стиля таблицы html
#Формат заголовков
style_zag = {'border': '1px solid #6E6E6E',
             'padding': '7px',
             'textAlign': 'center',
             'background-color': '#ABABAB',
             'text-shadow': '1px 1px 0 #fff'};

#Формат основной части
style_osn = {'border': '1px solid #c9c9c9',
             'padding': '3px 15px',
             'width': '95px',
             'textAlign': 'right'};
style_osn_2 = style_osn.copy();
style_osn['background-color'] = '#FFF';

#Формат столбца дат
style_d = {'border': '1px solid #6E6E6E',
           'padding': '3px 10px',
           'width': '15px',
           'textAlign': 'center',
           'background-color': '#C2C2C2'};

#Формат всей таблицы
style_tabl = {'border-collapse': 'collapse'};

def srvz_tabl():
    """
    Таблица данных помесячных цен.
    На входе фрейм данных.
    На выходе html дерево.
    """
    srvz_price = price_month_data.loc[:t_day.strftime('%m.%y')];
    srvz_price = srvz_price.iloc[-long_table:];

    d_head = [html.Tr([html.Td()] + 
                      [html.Th(col, style=style_zag) for col in srvz_price.columns])
             ];
    
    col = '';
    
    def stroka_tabl(i,ind,col):
        return html.Td(srvz_price.loc[i, col],
                        style=[style_osn, style_osn_2][ind%2]); #ind%2 позволяет поочередно использовать разные стили для столбцов
                         
    d_body = [html.Tr([html.Th(i, style=style_d)] + [stroka_tabl(i,ind,col) for ind,col in enumerate(srvz_price.columns)]
                     ) for i in srvz_price.index];
                   
    tablica = html.Table([html.Thead(d_head),
                          html.Tbody(d_body),
                         ], style=style_tabl);

    return html.P(tablica,
                  style={'padding-left': '50px'}
                 );

In [11]:
def empty_box():
    """
    Создание пустого элемента, чтобы начать новые элементы с новой строки.
    """
    empty_style = {'width': '100%'};
    
    return html.P([html.Label('')],  style = empty_style);

In [12]:
def serve_layout():
    """
    Собираем все части дашборда в один.
    """
    load_data(); #Загрузка данных
    
    obnov_lay = html.Div(style = {'backgroundColor': set_colors['backgroundColor']},
                         children = [dcc.Interval(id = 'interval-component', 
                                                  interval = 1, 
                                                  n_intervals = 0),

                                     nazv(), #Название
                                     graph(), #Графическая часть
                                     slider(), #Ползунок
                                     country(), #Выпадающий список стран
                                     #empty_box(), #Пустой бокс оставил на всяки случай
                                     region(), #Выпадающий список регионов
                                     peretok(), #Выпадающий список перетока
                                     naz_tabl(), #Название таблицы средневзвешенной цены
                                     srvz_tabl() #Таблица средневзвешенной цены
                                    ]
                        );
    return obnov_lay

In [13]:
app = dash.Dash(__name__);
app.title = 'NordPool';
app.layout = serve_layout;

#Возврат значений в дашборд
@app.callback(Output('plot', 'figure'),
              [Input('slider', 'value'),
               Input('country', 'value'),
               Input('region', 'value'),
               Input('peretok', 'value')])

def update_figure(slider_inp, country_list, region_value, peretok_list):
    """
    На входе: активные элементы дашборда.
    Используя показатели формируем новый фрейм.
    Отдаем новую подложку графика.
    """
    #Обновляем данные графика после смещения ползунков по дате
    create_graph(slider_inp[0], slider_inp[1]);
            
    #Определим какой регион выбран
    region_list = [];
    if region_value=='Все':
        for r in sostav_reg.values():
            region_list += r;
    elif region_value is not None:
        region_list = sostav_reg[region_value];
    
    #Обновляем видимости стран из выпадающего списка
    for nabor in data_graph:
        if nabor['name'] in country_list+region_list+peretok_list:
            nabor['visible'] = 'True';
        else:
            nabor['visible'] = 'legendonly';
            
    #Обновим границы осей 0y для отображаемых данных
    max_list, min_list = [[]]*2;
    for nabor in data_graph:
        if  nabor['visible'] == 'True':
            mnojitel = 10; #множитель для цен
            
            if 'Переток' in nabor['name']:
                mnojitel = 1; #множитель для перетока
                
            max_list.append(np.nanmax(nabor['y'])*mnojitel);
            min_list.append(np.nanmin(nabor['y'])*mnojitel);
    
    if len(max_list) > 0 and len(min_list) > 0:
        #Верхняя и нижняя граница
        verh, niz = max(0, np.nanmax(max_list)), min(0, np.nanmin(min_list));
        verh, niz = np.ceil(verh/100)*100,  np.floor(niz/100)*100;
        y_ax['yaxis2']['range'] = [niz/10, verh/10];
        y_ax['yaxis1']['range'] = [niz, verh];

    return graph().figure; #Возвращаем обновленную подложку графика после манипуляций на странице дашборда

if __name__ == '__main__':
    app.run_server(port=27016, host='10.79.237.204', debug=False);

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://10.79.237.204:27016/ (Press CTRL+C to quit)
10.79.98.24 - - [19/May/2022 10:16:00] "GET / HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:00] "GET /_dash-dependencies HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:00] "GET /_favicon.ico?v=1.8.0 HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:02] "GET /_dash-layout HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:02] "POST /_dash-update-component HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:22] "POST /_dash-update-component HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:26] "POST /_dash-update-component HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:28] "POST /_dash-update-component HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:30] "POST /_dash-update-component HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:34] "POST /_dash-update-component HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:36] "POST /_dash-update-component HTTP/1.1" 200 -
10.79.98.24 - - [19/May/2022 10:16:39] "PO